In [1]:
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
# from langchain.storage import InMemoryStore # Removed for direct Chroma inspection
from langchain_core.documents import Document
import os

persist_directory = './textbook_test_example1/'
id_key = "doc_id"

In [2]:
# Initialize Chroma vectorstore for summaries
vectorstore_summaries = Chroma(collection_name="summaries", persist_directory=persist_directory, embedding_function=GPT4AllEmbeddings())
# Initialize Chroma vectorstore for original texts
vectorstore_original_texts = Chroma(collection_name="original_texts", persist_directory=persist_directory, embedding_function=GPT4AllEmbeddings())


# Handling User Input: Analysis Image

In [3]:
query = "What's going on with my cat? What should I do?" 
image_path = "./cat.jpg"

import base64
import ollama # Ensure ollama library is installed (pip install ollama)

# --- Configuration for the image ---
# IMPORTANT: Adjust this path if your cat.jpg is in a different location
image_model = "minicpm-v:8b" # Or "llava:7b" or another suitable vision model you have installed via Ollama

# --- 1. Generate a textual summary of the image using an LLM ---
print(f" ⏳ Processing image: {image_path}")

image_summary = "Could not generate image summary." # Default in case of error
if not os.path.exists(image_path):
    print(f"Error: Image file not found at {image_path}. Please check the path.")
else:
    try:
        # Read and encode image in base64
        with open(image_path, 'rb') as f:
            image_data = base64.b64encode(f.read()).decode('utf-8')

        # Updated prompt for detailed image summarization
        image_summarization_prompt = """From a feline veterinary stand point, provide a highly detailed and objective 
                description of the image. Focus on all observable elements, actions, 
                objects, subjects, their attributes (e.g., color, size, texture), 
                their spatial relationships, and any discernible context or implied scene. 
                Describe any text present in the image. This description must be exhaustive 
                and purely factual, capturing every significant visual detail to serve as a 
                comprehensive textual representation for further analysis by another AI model. 
                If the image is entirely irrelevant or contains no discernible subject, 
                state "No relevant visual information."."""

        # Send image to ollama for vision model processing
        response = ollama.chat(
            model=image_model,
            messages=[
                {
                    'role': 'user',
                    'content': image_summarization_prompt,
                    'images': [image_data]
                }
            ]
        )
        image_summary = response['message']['content']
        print("--- Generated Image Summary ---")
        print(image_summary)

    except Exception as e:
        print(f"Error processing image with Ollama: {e}")

# This 'image_summary' can now be used along with your user's text query
# for retrieval or further processing in your RAG pipeline.

 ⏳ Processing image: ./cat.jpg
--- Generated Image Summary ---
The image depicts an adult cat with visible signs of distress and possible illness. The feline has prominent fur that appears matted in several areas, particularly around its neck and chest, indicating it may have been through a stressful event or is unwell. Its coat coloration includes brown, black, and white patches.

Its eyes are wide open, which can sometimes indicate alertness, anxiety, or discomfort. The cat's ears are perked up but not in an aggressive stance; this could suggest curiosity rather than aggression. Notably, the cat has a patch of fur that seems to be matted or missing, revealing pink skin underneath.

The feline is surrounded by pet supplies: there is a food bowl with what appears to be dry kibble and another container possibly filled with wet food or supplements; a blue water dish suggesting hydration provision for the animal. There's also an empty metallic feeding pan which could have been used earlie

# Handling User Input: Refine Query

In [4]:
import uuid
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Define the LLM for query refinement (using the same model as your RAG chain if appropriate)
query_refinement_model = ChatOllama(model="llama3.2")

# Prompt for query refinement
query_refinement_prompt = ChatPromptTemplate.from_template(
    """You are an intelligent assistant. Your task is to rephrase and expand the given user query \
into a more detailed and context-rich query that can be used to retrieve relevant information \
from a veterinary knowledge base. Use the provided image description to add visual context \
and relevant keywords to the refined query. Focus on adding relevant keywords, clarifying intent, \
and anticipating related information that might be helpful. The output should be a single, refined query.

Original query: {original_query}
Image description: {image_summary}"""
)

# Create the query refinement chain
query_refinement_chain = (
    {
        "original_query": RunnablePassthrough(),
        "image_summary": RunnablePassthrough()
    }
    | query_refinement_prompt
    | query_refinement_model
    | StrOutputParser()
)# --- Demonstration of query refinement and then retrieval with scores ---


print(f"Original user query: {query}")
print(f"Image Summary: {image_summary}")

refined_query = query_refinement_chain.invoke(
    {"original_query": query, "image_summary": image_summary}
)

print("-"*80)
print(f"Refined query: {refined_query}")


Original user query: What's going on with my cat? What should I do?
Image Summary: The image depicts an adult cat with visible signs of distress and possible illness. The feline has prominent fur that appears matted in several areas, particularly around its neck and chest, indicating it may have been through a stressful event or is unwell. Its coat coloration includes brown, black, and white patches.

Its eyes are wide open, which can sometimes indicate alertness, anxiety, or discomfort. The cat's ears are perked up but not in an aggressive stance; this could suggest curiosity rather than aggression. Notably, the cat has a patch of fur that seems to be matted or missing, revealing pink skin underneath.

The feline is surrounded by pet supplies: there is a food bowl with what appears to be dry kibble and another container possibly filled with wet food or supplements; a blue water dish suggesting hydration provision for the animal. There's also an empty metallic feeding pan which could h

# Query Decomposition

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
# Prompt for query decomposition
query_decomposition_prompt = ChatPromptTemplate.from_template(
    """You are an intelligent assistant. Your task is to break down the given complex query \
into a list of simpler, focused sub-queries. Each sub-query should be a standalone question \
that can be used to retrieve specific information from a veterinary knowledge base. \
Present the sub-queries as a JSON array of strings, where each string is a sub-query. \

Complex query: {refined_query}"""
)

# Create the query decomposition chain
query_decomposition_chain = (
    query_decomposition_prompt  # Prompt first
    | query_refinement_model    # Then the model
    | JsonOutputParser() # Use JsonOutputParser instead of StrOutputParser() | json.loads
)

# --- Demonstration of query decomposition ---

print(f"Original refined query: {refined_query[:300]} ....")

decomposed_queries = query_decomposition_chain.invoke({"refined_query": refined_query})

print("-" * 80)
print(f"Decomposed queries:\n{decomposed_queries}")

Original refined query: Refined Query:

"I'm concerned about my adult cat's sudden distress and possible illness, characterized by matted fur around its neck and chest, wide open eyes indicating alertness, anxiety, or discomfort, and perked-up ears. The cat's coat displays brown, black, and white patches. I'd like to know  ....
--------------------------------------------------------------------------------
Decomposed queries:
Here is the broken down list of simpler sub-queries:

1. What are common feline illnesses that can cause vomiting, diarrhea, lethargy, or difficulty breathing, characterized by matted fur around the neck and chest?

2. Are there any underlying medical conditions in cats that can lead to poor grooming habits resulting in matted fur, such as skin allergies or gastrointestinal issues?

3. What are recent stressors that may have contributed to a cat's sudden distress, including changes in environment, new pets, or other potential triggers?

4. How often should I prov